In [1]:
import os
from tqdm import tqdm

def getEpisodeLink(epnum):
    if epnum < 10:
        epnum_s = '0' + str(epnum)
    else:
        epnum_s = str(epnum)
    
    if epnum < 100:
        url = 'https://traffic.libsyn.com/secure/forcedn/seenunseen/E' + epnum_s + 'TheSeenandTheUnseen.mp3'
    else:
        url = 'https://traffic.libsyn.com/secure/forcedn/seenunseen/E' + epnum_s + 'SeenUnseen.mp3'
    
    return url

def getReducedName(fname):
    basename = os.path.basename(fname)
    new_basename = 'E'
    for char in basename[1:]:
        if char.isdigit():
            new_basename += char
        else:
            break
    return fname.replace(basename, new_basename) + '.mp3'



In [5]:
ep_num = 310
url = getEpisodeLink(ep_num)
reduced_name = 'E' + str(ep_num) + '.mp3'
reduced_name_fullpath = os.path.join('data/seenunseen_mp3', reduced_name)

In [9]:
!wget -O {reduced_name_fullpath} {url}

--2023-01-02 15:44:12--  https://traffic.libsyn.com/secure/forcedn/seenunseen/E310SeenUnseen.mp3
Resolving traffic.libsyn.com (traffic.libsyn.com)... 204.16.246.216
Connecting to traffic.libsyn.com (traffic.libsyn.com)|204.16.246.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://content.libsyn.com/p/a/6/9/a695582d4604ed3a/E310SeenUnseen.mp3?c_id=143496897&response-content-disposition=attachment&cs_id=143496897&response-content-type=audio%2Fmpeg&Expires=1672695377&Signature=IIldOxI4RZpCbthI-Xl3q4fBKpk2w3PRgYAVMrpaOArzJeN5J0sr4glmR4nd64RMJCakhdShxCXlbXSbY~TbK651itFkWLQhTwQpNKQdIbqJlEHfXKNbWVvKgzTW3rQEyLLF8w9g8~Id8nbPoXu42iTTCFY-qZ0ArukHoQfVq1BuTv8S-je7s7Q5CyrbnYTQIoV-BGhm3GdwbUMbm4pmoDFW0d-Hq57X6KNM1p5PWg3N5HfcQsONvJ0LcTSiiJYA6~TDpEjBzlgOZ2nlrGKFX8izzPy2EnyQ1MS5vFOhD3TjSSq62nDfTdqW8KSyvZdC7Z7To24bNHeeSziIat2Wsg__&Key-Pair-Id=K1YS7LZGUP96OI [following]
--2023-01-02 15:44:13--  https://content.libsyn.com/p/a/6/9/a695582d4604ed3a/E310SeenUnseen.mp3?c

In [11]:
# see if downloaded
! du -sh {reduced_name_fullpath}

243M	data/seenunseen_mp3/E310.mp3


In [15]:
# run whisper on it
run = "bash run_whisper.sh " + reduced_name_fullpath


In [16]:
!{run}

[00:00.000 --> 00:10.920]  2022 has just come to an end, and so will we. Given our mortality, it's astonishing
[00:10.920 --> 00:15.720]  that we are such an arrogant species. We behave, at an individual and collective level, as
[00:15.720 --> 00:20.000]  if we are immortal. We know almost nothing about the vast and complex universe around
[00:20.000 --> 00:24.920]  us, but we have full conviction in the stories we tell ourselves about it. And we are just
[00:24.920 --> 00:29.960]  beginning to get to grips with our own bodies. For most of human history, we knew so little
[00:29.960 --> 00:34.400]  about the flesh and bones that we are made of. Not long ago, we even thought the brain
[00:34.400 --> 00:39.800]  was a part of the heart. Now, while our arrogance remains as unjustified as before, we have
[00:39.800 --> 00:44.120]  more reason to hope. Science has brought us to a place where we understand our bodies
[00:44.120 --> 00:48.680]  better. We understand our different organs, the 

In [17]:
# convert mp3 to wav for diarization input
infile = reduced_name_fullpath
outfile = reduced_name_fullpath.replace('mp3', 'wav')

!ffmpeg -loglevel quiet -i {infile} -acodec pcm_s16le -ac 1 -ar 16000 {outfile}

In [21]:
# now diarize
fname = reduced_name_fullpath.replace('mp3', 'wav')
run = "bash run_pyannote.sh " + fname
!{run}

Doing file data/seenunseen_wav/E310.wav


In [22]:
# combine diarization and whisper output
from utils import *

fname = reduced_name
subs = getProcessedSubs(fname)
out_file = os.path.join('data/seenunseen_transcripts_diarized', fname + '.vtt')
subs.save(out_file)